<a href="https://colab.research.google.com/github/rexad/NLP_notebooks/blob/master/Kaggle/jigsaw/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#logistic regression with Scikit-learn

##Importing our libraries

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence

import gensim.downloader as api
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

import os.path
from google.colab import drive

##Load data for jigsaw 

In [14]:
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/"
train = pd.read_csv('/content/gdrive/My Drive/Data/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Data/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'   Data					 ML_Training
 Colab_Notebooks   'Machine Learning A-Z (Codes and Datasets)'


##Feature engineering

###Load embeddings using gensim library 

In [15]:
base_path_embed = '/content/gdrive/My Drive/Data/Embeddings/'
embeddings = [
              'fasttext-wiki-news-subwords-300',
              'glove-twitter-100',
              'glove-wiki-gigaword-300'
              ]
for embed in embeddings:
  if os.path.isfile(base_path_embed+embed) == False:
    embed_file = api.load(embed)
    embed_file.save(base_path_embed+embed)
    del embed_file

###Extract text information and tokenize

In [18]:
spcial_carracter = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None, split= ' ', filters = spcial_carracter
)
tokenizer.fit_on_texts(list(train['comment_text']) + list(test['comment_text']))
x_train = tokenizer.texts_to_sequences(train['comment_text'])
x_test = tokenizer.texts_to_sequences(test['comment_text'])
x_train = sequence.pad_sequences(x_train, maxlen=220)
x_test = sequence.pad_sequences(x_test, maxlen=220)
y_train = np.where(train['target'] > 0.5, 1, 0)

###Convert words to embeddigns

In [21]:
def build_embed_matrix(embed, tokens):

  embed_vec = KeyedVectors.load(base_path_embed + embed, mmap='r')
  embed_matrix = np.zeros((len(tokens) + 1,300))
  for i, word in tokens.items():
    for candidate in [word, word.lower()]:
      if candidate in embed_vec:
        embed_matrix[i]= embed_vec[candidate]
  return embed_matrix

In [22]:
embed_matrix = build_embed_matrix('fasttext-wiki-news-subwords-300', tokenizer.index_word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [37]:
x_train.shape

(1804874, 220)

In [38]:
train['comment_text'].shape

(1804874,)